# 3.0 — Match & Merge Base de Dados

**Objetivo:** Realizar a integração entre a base Lighthouse e a base Kaggle (movies_metadata) a partir de chaves de correspondência. O processo de casamento será feito prioritariamente por (title_norm, Year) e, em caso de falha, com fallback para (original_title_norm, Year).

**Produtos gerados:**

- data/processed/lighthouse_enriched.csv → base unificada e enriquecida;

- data/processed/unmatched_lighthouse.csv → registros da Lighthouse sem correspondência;

- data/processed/match_diagnostics.csv → amostra para diagnóstico do processo de matching.

**Saída principal:** Arquivo enriquecido em `data/processed/lighthouse_enriched.csv`

## 3.1 - Imports

In [1]:
# =====================================================
# Imports
# =====================================================

# Manipulação e análise de dados
import math
import numpy as np
import pandas as pd
import re
from pathlib import Path

# Visualização de dados
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from tabulate import tabulate

# Sistema e paths
import os
from pathlib import Path

# Utilidades para notebooks
from IPython.display import display
from IPython.display import Image
from IPython.core.display import HTML

# Manipulação de datas
import datetime

# Supress warnings
import warnings

In [2]:
warnings.filterwarnings('ignore')

## 3.2 - Funções Auxiliares

In [3]:
# ==== Caminhos & Display ====
class PATHS:
    """
    Caminhos padrão do projeto quando o notebook roda dentro de 'notebooks/'.

    Regra:
    - Se o cwd termina com 'notebooks', a raiz é o pai (../)
    - Caso contrário, assume que já estamos na raiz (útil se abrir o Jupyter na raiz)
    """
    _CWD = Path.cwd()
    ROOT = _CWD.parent if _CWD.name == "notebooks" else _CWD
    RAW  = ROOT / "data" / "raw"
    PROC = ROOT / "data" / "processed"
    INTER = ROOT / "data" / "intermediary"
    REP  = ROOT / "reports"
    FIG  = REP / "figures"

def ensure_dirs():
    """Garante que as pastas principais existam."""
    for p in [PATHS.RAW, PATHS.INTER, PATHS.PROC, PATHS.REP, PATHS.FIG]:
        p.mkdir(parents=True, exist_ok=True)

def set_display(max_cols: int = 100, decimals: int = 2):
    """Ajusta visualização padrão no Pandas/Seaborn para leitura eficiente."""
    pd.options.display.max_columns = max_cols
    fmt = "{:." + str(decimals) + "f}"
    pd.options.display.float_format = fmt.format
    sns.set(style="whitegrid", palette="muted", font_scale=1.1)

# ==== Parsing & Normalização ====
def normalize_title(s):
    """Normaliza título: minúsculas, sem pontuação, sem espaços extras."""
    if pd.isna(s):
        return np.nan
    s = s.lower()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^\w\s]", "", s)
    return s.strip()

def parse_year(x):
    """Extrai o primeiro ano encontrado (ex.: '1994' ou '1994(I)')."""
    if pd.isna(x):
        return pd.NA
    m = re.search(r"\d{4}", str(x))
    return int(m.group()) if m else pd.NA

def to_numeric_safe(series):
    """Converte para numérico com 'coerce' (incompatíveis viram NaN)."""
    return pd.to_numeric(series, errors="coerce")

# ==== Deduplicação ====
def dedup_by_key(df, keys, score_col="vote_count"):
    """
    Se houver múltiplas linhas para a mesma chave, mantém a de maior `score_col`.
    Ex.: ao agrupar por (title_norm, Year), fica a linha com maior número de votos.
    """
    tmp = df.copy()
    tmp[score_col] = to_numeric_safe(tmp[score_col]) if score_col in tmp.columns else 0
    tmp = tmp.sort_values(by=[score_col], ascending=False)
    return tmp.drop_duplicates(subset=keys, keep="first")

In [4]:
# Chamo as de caminhos e display
ensure_dirs()
set_display()

## 3.3 - Carregando os Dados e Realizando checagens

In [5]:
# Realizo a leitura dos dados através da classe
# Lighthouse pré-limpa
lh = pd.read_csv(PATHS.INTER / "lighthouse_clean.csv", low_memory=False)

# Chaves da base Kaggle
km_t = pd.read_csv(PATHS.INTER / "kaggle_movies_key_title_year.csv", low_memory=False)           # (title_norm, Year)
km_o = pd.read_csv(PATHS.INTER / "kaggle_movies_key_originaltitle_year.csv", low_memory=False)   # (original_title_norm, Year)

In [6]:
# Lighthouse
lh["Year"] = pd.to_numeric(lh["Year"], errors="coerce").astype("Int64")

# Kaggle (título normalizado)
km_t["Year"] = pd.to_numeric(km_t["Year"], errors="coerce").astype("Int64")

# Kaggle (título original normalizado)
km_o["Year"] = pd.to_numeric(km_o["Year"], errors="coerce").astype("Int64")

In [7]:
# Realizo checagens rápidas
print("Lighthouse Metabase:", lh.shape)
print("Kaggle Metabase:", km_t.shape, km_o.shape)

Lighthouse Metabase: (999, 19)
Kaggle Metabase: (45376, 27) (45411, 27)


In [8]:
display(lh.head(3))
display(km_t.head(3))
display(km_o.head(3))

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Year,Runtime_min,Gross_USD,title_norm
0,The Godfather,1972,A,175 min,"Crime, Drama",9.20,An organized crime dynasty's aging patriarch t...,100.00,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",1972,175.00,134966411.00,the godfather
1,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.00,When the menace known as the Joker wreaks havo...,84.00,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",2008,152.00,534858444.00,the dark knight
2,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.00,The early life and career of Vito Corleone in ...,90.00,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",1974,202.00,57300000.00,the godfather part ii


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,Year,title_norm,original_title_norm
0,False,NaN,160000000.00,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",http://inceptionmovie.warnerbros.com/,27205,tt1375666,en,Inception,"Cobb, a skilled thief who commits corporate es...",29.11,/qmDpIHrmpJINaRKAfWQfftjCdyi.jpg,"[{'name': 'Legendary Pictures', 'id': 923}, {'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2010-07-14,825532764.00,148.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Your mind is the scene of the crime.,Inception,False,8.10,14075.00,2010,inception,inception
1,False,"{'id': 263, 'name': 'The Dark Knight Collectio...",185000000.00,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",http://thedarkknight.warnerbros.com/dvdsite/,155,tt0468569,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.17,/1hRoyzDtpgMU7Dz4JF22RANzQO7.jpg,"[{'name': 'DC Comics', 'id': 429}, {'name': 'L...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2008-07-16,1004558444.00,152.00,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Why So Serious?,The Dark Knight,False,8.30,12269.00,2008,the dark knight,the dark knight
2,False,"{'id': 87096, 'name': 'Avatar Collection', 'po...",237000000.00,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.07,/kmcqlZGaSh20zpTbuoF0Cdn07dT.jpg,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,2787965087.00,162.00,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.20,12114.00,2009,avatar,avatar


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,Year,title_norm,original_title_norm
0,False,NaN,160000000.00,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",http://inceptionmovie.warnerbros.com/,27205,tt1375666,en,Inception,"Cobb, a skilled thief who commits corporate es...",29.11,/qmDpIHrmpJINaRKAfWQfftjCdyi.jpg,"[{'name': 'Legendary Pictures', 'id': 923}, {'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2010-07-14,825532764.00,148.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Your mind is the scene of the crime.,Inception,False,8.10,14075.00,2010,inception,inception
1,False,"{'id': 263, 'name': 'The Dark Knight Collectio...",185000000.00,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",http://thedarkknight.warnerbros.com/dvdsite/,155,tt0468569,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.17,/1hRoyzDtpgMU7Dz4JF22RANzQO7.jpg,"[{'name': 'DC Comics', 'id': 429}, {'name': 'L...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2008-07-16,1004558444.00,152.00,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Why So Serious?,The Dark Knight,False,8.30,12269.00,2008,the dark knight,the dark knight
2,False,"{'id': 87096, 'name': 'Avatar Collection', 'po...",237000000.00,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,tt0499549,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.07,/kmcqlZGaSh20zpTbuoF0Cdn07dT.jpg,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,2787965087.00,162.00,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,False,7.20,12114.00,2009,avatar,avatar


In [9]:
# Verifico se todas as colunas-chaves estão presentes
assert {"title_norm","Year"}.issubset(lh.columns)
assert {"title_norm","Year"}.issubset(km_t.columns)
assert {"original_title_norm","Year"}.issubset(km_o.columns)

## 3.4 - Match 1: chave principal (title_norm, Year)

In [10]:
# Renomeio as colunas prefixando para evitar colisão de nomes com a lh (exceto imdb_id, title e original_title)
rename_map = {
    "id": "kg_id",
    "budget": "kg_budget",
    "revenue": "kg_revenue",
    "runtime": "kg_runtime",
    "vote_average": "kg_vote_avg",
    "vote_count": "kg_vote_count",
    "release_date": "kg_release_date",
    "original_language": "kg_lang",
    "genres": "kg_genres",
    "production_companies": "kg_prod_companies",
    "production_countries": "kg_prod_countries",
    "spoken_languages": "kg_spoken_languages",
    "homepage": "kg_homepage",
    "tagline": "kg_tagline",
    "popularity": "kg_popularity"
}

km_t = km_t.rename(columns=rename_map)
km_o = km_o.rename(columns=rename_map)

print("Colunas Kaggle normalizadas (km_t):", km_t.columns[:15])
print("Colunas Kaggle normalizadas (km_o):", km_o.columns[:15])


Colunas Kaggle normalizadas (km_t): Index(['adult', 'belongs_to_collection', 'kg_budget', 'kg_genres',
       'kg_homepage', 'kg_id', 'imdb_id', 'kg_lang', 'original_title',
       'overview', 'kg_popularity', 'poster_path', 'kg_prod_companies',
       'kg_prod_countries', 'kg_release_date'],
      dtype='object')
Colunas Kaggle normalizadas (km_o): Index(['adult', 'belongs_to_collection', 'kg_budget', 'kg_genres',
       'kg_homepage', 'kg_id', 'imdb_id', 'kg_lang', 'original_title',
       'overview', 'kg_popularity', 'poster_path', 'kg_prod_companies',
       'kg_prod_countries', 'kg_release_date'],
      dtype='object')


In [11]:
# Seleciono só colunas Kaggle que serão incorporadas ao lh
cols_kaggle = [
    "kg_id","imdb_id","kg_release_date",
    "kg_budget","kg_revenue","kg_runtime",
    "kg_vote_avg","kg_vote_count","kg_lang",
    "kg_genres","kg_prod_companies","kg_prod_countries",
    "kg_spoken_languages","kg_homepage","kg_tagline","kg_popularity"
]
cols_kaggle = [c for c in cols_kaggle if c in km_t.columns]

m1 = lh.merge(
    km_t[["title_norm","Year"] + cols_kaggle],
    on=["title_norm","Year"],
    how="left",
    suffixes=("", "_km1")
)

match_rate_1 = m1["kg_id"].notna().mean()
print(f"[MATCH 1] Cobertura via (title_norm, Year): {match_rate_1:.1%}")

[MATCH 1] Cobertura via (title_norm, Year): 73.5%


## 3.5- Match 2(fallback):

In [12]:
not_matched = m1["kg_id"].isna()

m2 = m1.loc[not_matched, ["Series_Title","Year","title_norm"]].merge(
    km_o[["original_title_norm","Year"] + cols_kaggle],
    left_on=["title_norm","Year"],
    right_on=["original_title_norm","Year"],
    how="left"
)

# Preenche m1 apenas onde não tinha match
m_final = m1.copy()
for c in cols_kaggle:
    m_final.loc[not_matched, c] = m2[c].values

# Marca qual chave casou
m_final["_matched_key"] = np.where(
    m_final["kg_id"].notna(),
    np.where(m1["kg_id"].notna(), "title_norm", "original_title_norm"),
    pd.NA
)

# Métricas
match_rate_total = m_final["kg_id"].notna().mean()
match_via_fallback = ((m1["kg_id"].isna()) & (m_final["kg_id"].notna())).mean()

print(f"[MATCH 2] Ganho via fallback (original_title_norm, Year): {match_via_fallback:.1%}")
print(f"[TOTAL ] Cobertura final (qualquer chave): {match_rate_total:.1%}")

[MATCH 2] Ganho via fallback (original_title_norm, Year): 8.7%
[TOTAL ] Cobertura final (qualquer chave): 82.2%


In [13]:
# # Conversão explícita de tipos nas colunas críticas
# m_final["Year"]         = m_final["Year"].astype("Int64")   # ano pode ter NaN
# m_final["kg_budget"]    = m_final["kg_budget"].astype(float)
# m_final["kg_revenue"]   = m_final["kg_revenue"].astype(float)
# m_final["kg_runtime"]   = m_final["kg_runtime"].astype(float)
# m_final["kg_vote_count"] = m_final["kg_vote_count"].astype("Int64")

In [14]:
# Checagem
# Conversões úteis para leitura (em milhões)
for raw, pretty in [("kg_budget","kg_budget_musd"), ("kg_revenue","kg_revenue_musd")]:
    if raw in m_final.columns:
        m_final[pretty] = pd.to_numeric(m_final[raw], errors="coerce") / 1e6

# Visão rápida dos que casaram
cols_view = [
    "Series_Title","Year","IMDB_Rating","Meta_score","No_of_Votes",
    "kg_id","imdb_id","kg_release_date","kg_budget_musd","kg_revenue_musd",
    "kg_vote_avg","kg_vote_count","_matched_key"
]
cols_view = [c for c in cols_view if c in m_final.columns]
display(m_final.loc[m_final["kg_id"].notna(), cols_view].head(10))


,Series_Title,Year,IMDB_Rating,Meta_score,No_of_Votes,kg_id,imdb_id,kg_release_date,kg_budget_musd,kg_revenue_musd,kg_vote_avg,kg_vote_count,_matched_key
0,The Godfather,1972,9.20,100.00,1620367,238.00,tt0068646,1972-03-14,6.00,245.07,8.50,6024.00,title_norm
1,The Dark Knight,2008,9.00,84.00,2303232,155.00,tt0468569,2008-07-16,185.00,1004.56,8.30,12269.00,title_norm
2,The Godfather: Part II,1974,9.00,90.00,1129952,240.00,tt0071562,1974-12-20,13.00,47.54,8.30,3418.00,title_norm
3,12 Angry Men,1957,9.00,96.00,689845,389.00,tt0050083,1957-03-25,0.35,1.00,8.20,2130.00,title_norm
4,The Lord of the Rings: The Return of the King,2003,8.90,94.00,1642758,122.00,tt0167260,2003-12-01,94.00,1118.89,8.10,8226.00,title_norm
5,Pulp Fiction,1994,8.90,94.00,1826188,680.00,tt0110912,1994-09-10,8.00,213.93,8.30,8670.00,title_norm
6,Schindler's List,1993,8.90,94.00,1213505,424.00,tt0108052,1993-11-29,22.00,321.37,8.30,4436.00,title_norm
7,Inception,2010,8.80,74.00,2067042,27205.00,tt1375666,2010-07-14,160.00,825.53,8.10,14075.00,title_norm
8,Fight Club,1999,8.80,66.00,1854740,550.00,tt0137523,1999-10-15,63.00,100.85,8.30,9678.00,title_norm
9,The Lord of the Rings: The Fellowship of the Ring,2001,8.80,92.00,1661481,120.00,tt0120737,2001-12-18,93.00,871.37,8.00,8892.00,title_norm


In [15]:
m_final.columns

Index(['Series_Title', 'Released_Year', 'Certificate', 'Runtime', 'Genre',
       'IMDB_Rating', 'Overview', 'Meta_score', 'Director', 'Star1', 'Star2',
       'Star3', 'Star4', 'No_of_Votes', 'Gross', 'Year', 'Runtime_min',
       'Gross_USD', 'title_norm', 'kg_id', 'imdb_id', 'kg_release_date',
       'kg_budget', 'kg_revenue', 'kg_runtime', 'kg_vote_avg', 'kg_vote_count',
       'kg_lang', 'kg_genres', 'kg_prod_companies', 'kg_prod_countries',
       'kg_spoken_languages', 'kg_homepage', 'kg_tagline', 'kg_popularity',
       '_matched_key', 'kg_budget_musd', 'kg_revenue_musd'],
      dtype='object')

In [16]:
m_final.shape

(999, 38)

In [17]:
m_final.dtypes

Series_Title            object
Released_Year           object
Certificate             object
Runtime                 object
Genre                   object
IMDB_Rating            float64
Overview                object
Meta_score             float64
Director                object
Star1                   object
Star2                   object
Star3                   object
Star4                   object
No_of_Votes              int64
Gross                   object
Year                     Int64
Runtime_min            float64
Gross_USD              float64
title_norm              object
kg_id                   object
imdb_id                 object
kg_release_date         object
kg_budget              float64
kg_revenue             float64
kg_runtime             float64
kg_vote_avg            float64
kg_vote_count          float64
kg_lang                 object
kg_genres               object
kg_prod_companies       object
kg_prod_countries       object
kg_spoken_languages     object
kg_homep

## 3.6 - Salvando saídas e diagnóstico

In [18]:
enriched_path  = PATHS.PROC / "lighthohouse_enriched.csv" # base de dados enriquecida
unmatched_path = PATHS.PROC / "unmatched_lighthouse.csv"
diag_path      = PATHS.PROC / "matching_diagnostics.csv" 

PATHS.PROC

# Base final enriquecida
m_final.to_csv(enriched_path, index=False)

# Filmes da Lighthouse que não casaram (para investigação manual)
m_final.loc[m_final["kg_id"].isna(), ["Series_Title","Year","title_norm"]].to_csv(unmatched_path, index=False)

# Diagnóstico: quais chaves casaram e principais colunas de conferência
diag_cols = ["Series_Title","Year","title_norm","kg_id","imdb_id","kg_release_date","kg_vote_avg","kg_vote_count","_matched_key"]
diag_cols = [c for c in diag_cols if c in m_final.columns]
m_final[diag_cols].to_csv(diag_path, index=False)

print("Salvos:")
print(" -", enriched_path.resolve())
print(" -", unmatched_path.resolve())
print(" -", diag_path.resolve())

Salvos:
 - /home/emersds/repos_projetos/project_lighthouse/data/processed/lighthohouse_enriched.csv
 - /home/emersds/repos_projetos/project_lighthouse/data/processed/unmatched_lighthouse.csv
 - /home/emersds/repos_projetos/project_lighthouse/data/processed/matching_diagnostics.csv


# 3.7 -  Decisões e Próximos passos

**Decisões nesta etapa**
- Usei title_norm + Year como chave principal de junção e original_title_norm + Year como fallback.
- Cobertura final do match ficou em 82,2% (73,5% pela chave principal + 8,7% via fallback).
- Padronizei os campos do Kaggle com prefixo kg_ para evitar colisão com as colunas da Lighthouse e facilitar a leitura (ex.: kg_budget_musd, kg_revenue_musd, kg_vote_avg).
- Mantive a coluna _matched_key para auditoria (saber por qual chave cada linha casou).
- Não forcei imputação nem correção manual de casos sem match nesta fase — a análise virá na EDA.

**Saídas desta etapa**
- data/processed/lighthouse_enriched.csv — base da Lighthouse enriquecida com colunas do Kaggle.
- data/processed/unmatched_lighthouse.csv — filmes da Lighthouse sem correspondência (para eventual revisão).
- data/processed/match_diagnostics.csv — amostra com campos‐chave para conferência rápida.

**Próximos passos**
- Como a base de dados expandida do Kaggle é significativamente maior e já contém mais de 80% dos registros da base lighthouse, optei por trabalhar com a abase Kaggle a aprtir de agora.
- Essa decisão se deve ao fato de que base de dados muito pequenas tendem a aumentar o risco de overfitting e não oferecem representatividade suficiente para treinar e avaliar modelos de forma robusta.
- Em uma próxima etapa (ciclo do projeto), pretendo utilizar a base lighthouse de forma comparativa, a fim de verificar a performance dos modelos em um cenário com menos dados e avaliar possíveis diferenças de generalização